# test_eldercare_api.ipynb

This notebook provides code to test connectivity to the ElderCare API, and demonstrates LLM tool use with the API. Before using this notebook you must create a username and password for the Eldercare API.

In [1]:
!pip install -qU zeep

In [1]:
from dotenv import load_dotenv
import os

load_dotenv('../app/.env')

ELDERCARE_API_USERNAME = os.getenv("ELDERCARE_API_USERNAME")
ELDERCARE_API_PASSWORD = os.getenv("ELDERCARE_API_PASSWORD")


Information about the Eldercare API can be found here: https://eldercare.acl.gov/WebServices/EldercareData/ec_search.asmx

In [2]:
from zeep import Client

# Initialize the client with the WSDL URL
wsdl = 'https://eldercare.acl.gov/WebServices/EldercareData/ec_search.asmx?WSDL'
client = Client(wsdl=wsdl)

# Call the login method
session_token = client.service.login(ELDERCARE_API_USERNAME, ELDERCARE_API_PASSWORD)
print(f"Logged in with session token {session_token}")


Logged in with session token 2d96b9fc-f19c-48b8-bbdd-55a273c504f1


In [27]:

# Use the session token to search by city and state
result = client.service.SearchByCityState(asCity="Jacksonville", asState="FL", asToken=session_token)

# Print the result
print(result)


{
    'schema': <Schema(location=None, tns=None)>,
    '_value_1': {
        '_value_1': [
            {
                'Table1': {
                    'AID': 85201,
                    'Name': 'ElderSource',
                    'AgencyType': 'AAA',
                    'LeadID': 8001,
                    'Director': 'Linda Levin',
                    'Address1': '10688 Old Saint Augustine Road',
                    'Address2': None,
                    'City': 'Jacksonville',
                    'StateCode': 'NV',
                    'County': None,
                    'ZipCode': '32257',
                    'ZipCode4': '    ',
                    'O_Phone': '904-391-6699',
                    'Info_Phone': '888-242-4464',
                    'NFT_Phone': '888-242-4464',
                    'SFT_Phone': None,
                    'RFT_Phone': None,
                    'TTY_Phone': '904-391-6697',
                    'FaxNum': None,
                    'EMailAdd': 'information@myelderso

In [28]:
# Call the SearchByZip method
result = client.service.SearchByZip(asZipCode="02421", asToken=session_token)

# Print the result
print(result)

{
    'schema': <Schema(location=None, tns=None)>,
    '_value_1': {
        '_value_1': [
            {
                'Table1': {
                    'tabName': 'Information and Assistance',
                    'AID': 17501,
                    'Name': 'Executive Office of Elder Affairs',
                    'AgencyType': 'SSP',
                    'LeadID': 155301,
                    'Director': None,
                    'Address1': '1 Ashburton Place',
                    'Address2': None,
                    'City': 'Boston',
                    'StateCode': 'MA',
                    'County': None,
                    'ZipCode': '02108',
                    'ZipCode4': '    ',
                    'O_Phone': '617-727-7750',
                    'Info_Phone': '800-243-4636',
                    'NFT_Phone': None,
                    'SFT_Phone': '800-243-4636',
                    'RFT_Phone': None,
                    'TTY_Phone': None,
                    'FaxNum': None,
       

Now work on incorporating these APIs into a LangGraph flow

In [29]:
!pip install -qU langgraph

In [3]:
from zeep import Client
from langchain_anthropic import ChatAnthropic
from langchain_core.tools import tool


# Initialize SOAP client
wsdl = 'https://eldercare.acl.gov/WebServices/EldercareData/ec_search.asmx?WSDL'
client = Client(wsdl=wsdl)

session_token = client.service.login(ELDERCARE_API_USERNAME, ELDERCARE_API_PASSWORD)
print(f"Logged in with session token {session_token}")

# Define tools
@tool
def search_by_city_state(city: str, state: str):
    """Uses the Eldercare Data API to search for elder care close to a given city and state"""
    return client.service.SearchByCityState(asCity=city, asState=state, asToken=session_token)

@tool
def search_by_zip(zip_code: str):
    """Uses the Eldercare Data API to search for elder care close to a zip code"""
    return client.service.SearchByZip(asZipCode=zip_code, asToken=session_token)

Logged in with session token f37dcc68-39f9-405d-8f94-3c73ff1a0450


In [4]:
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

llm_with_tools = ChatAnthropic(
    model="claude-3-haiku-20240307",    
    anthropic_api_key=ANTHROPIC_API_KEY,
).bind_tools([search_by_city_state, search_by_zip])

In [51]:
from zeep.helpers import serialize_object

def condense_tool_output(tool_output):
    # Serialize the Zeep complex object into a Python dictionary
    serialized_data = serialize_object(tool_output)

    # Initialize an empty list to hold the formatted agency information
    formatted_output = []

    # Iterate through the agencies in the serialized data
    for agency in serialized_data['_value_1']['_value_1']:
        info = agency.get('Table1', {})

        # Extract relevant fields and handle missing data
        name = info.get('Name', 'N/A')
        address = f"{info.get('Address1', '')}, {info.get('City', '')}, {info.get('StateCode', '')} {info.get('ZipCode', '')}".strip(", ")
        phone = info.get('O_Phone', 'N/A')
        email = info.get('EMailAdd', 'N/A')
        url = info.get('URL', 'N/A')
        description = info.get('Description', 'No description available')

        # Format each agency's info into a single line
        formatted_entry = f"Name: {name} | Address: {address} | Phone: {phone} | Email: {email} | URL: {url} | Description: {description}"

        # Append the formatted entry to the output list
        formatted_output.append(formatted_entry)

    # Join all the entries with a single newline between each
    return "\n".join(formatted_output)





In [59]:
from langchain_core.messages import HumanMessage, ToolMessage

async def search_result(messages):

    tool_call_results = ""

    ai_msg = await llm_with_tools.ainvoke(messages)
    for tool_call in ai_msg.tool_calls:
        selected_tool = {"search_by_city_state": search_by_city_state, "search_by_zip": search_by_zip}[tool_call["name"].lower()]
        tool_output = await selected_tool.ainvoke(tool_call["args"])
        tool_call_results += condense_tool_output(tool_output)

    print(tool_call_results)

    return tool_call_results




In [62]:
messages = [HumanMessage("what are some eldercare resources in Phoenix?")]
tool_output = await search_result(messages)

Name: Area Agency on Aging, Region One | Address: 1366 East Thomas Road, Phoenix, AZ 85014 | Phone: 602-264-2255 | Email: info@aaaphx.org | URL: http://www.aaaphx.org | Description: Serves adults 60 and older, family caregivers of older adults, adults with disabilities and long-term care needs, adults diagnosed with HIV/AIDS, and victims of late-life domestic violence, elder abuse, sexual assault and elder refugees.
Name: Area Agency on Aging, Region One | Address: 1366 East Thomas Road, Phoenix, AZ 85014 | Phone: 602-264-2255 | Email: info@aaaphx.org | URL: http://www.aaaphx.org | Description: Serves adults 60 and older, family caregivers of older adults, adults with disabilities and long-term care needs, adults diagnosed with HIV/AIDS, and victims of late-life domestic violence, elder abuse, sexual assault and elder refugees.
Name: Area Agency on Aging, Region One | Address: 1366 East Thomas Road, Phoenix, AZ 85014 | Phone: 602-264-2255 | Email: info@aaaphx.org | URL: http://www.aaap

In [63]:
messages = [HumanMessage("why is the sky blue?")]
tool_output = await search_result(messages)

In [66]:
messages = [HumanMessage("what is sundowning?")]
tool_output = await search_result(messages)

In [65]:
messages = [HumanMessage("where can i find adult daycare in 02421?")]
tool_output = await search_result(messages)

Name: Executive Office of Elder Affairs | Address: 1 Ashburton Place, Boston, MA 02108 | Phone: 617-727-7750 | Email: elder.affairs@state.ma.us | URL: https://www.massoptions.org/massoptions/ | Description: MassOptions will find the local organization that can best meet your needs.
Name: Minuteman Senior Services | Address: One Burlington Woods Drive, Burlington, MA 01803 | Phone: 781-272-7177 | Email: elderinfo@minutemansenior.org | URL: https://minutemansenior.org | Description: None
Name: Minuteman Senior Services | Address: One Burlington Woods Drive, Burlington, MA 01803 | Phone: 781-272-7177 | Email: elderinfo@minutemansenior.org | URL: https://minutemansenior.org | Description: None
Name: Massachusetts Executive Office of Elder Affairs | Address: 1 Ashburton Place, Boston, MA 02108 | Phone: None | Email: information.resources@mass.gov | URL: https://www.mass.gov/orgs/executive-office-of-elder-affairs | Description: State Agency that contracts with local service providers.
Name: 